## csv 파일 concat 전 전처리 코드

1. 각 파일에서 연도 추출하여 GameID에 붙이기
2. 날짜 변환 및 정렬
3. 문자형 컬럼을 숫자형 컬럼으로 변환
4. 승/패/무 + 홈/어웨이 정보 저장
5. 무승부 제거 (이진 분류 예측을 위함)
6. 팀 이름 숫자로 인코딩

In [ ]:
import pandas as pd

def preprocess_baseball_data_and_save(file_paths, output_path):
    # 여러 csv 파일을 읽어 하나의 DataFrame으로 합침
    df_total = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

    # 각 파일에서 연도 추출하여 GameID에 붙이기
    for fp in file_paths:
        year = fp.split('_')[0]
        mask = df_total['GameID'].isin(pd.read_csv(fp)['GameID'])
        df_total.loc[mask, 'GameID'] = year + '-' + df_total.loc[mask, 'GameID']

    # 날짜 변환 및 정렬
    df_total['GameDate'] = pd.to_datetime(df_total['GameID'], errors='coerce')


    # 문자열 분할하여 숫자형 컬럼으로 변환
    df_total[['GO', 'FO']] = df_total['GO-FO'].str.split('-', expand=True).astype(int)
    df_total[['NP_total', 'NP_strike']] = df_total['NP-S'].str.split('-', expand=True).astype(int)
    df_total[['IR', 'IS']] = df_total['IR-IS'].str.split('-', expand=True).astype(int)
    df_total.drop(columns=['GO-FO', 'NP-S', 'IR-IS'], inplace=True)

    # 승/패/무 생성 + 홈/어웨이 정보 저장
    results = []
    home_away = []
    for i in range(0, len(df_total), 2):
        r1 = df_total.loc[i, 'R']
        r2 = df_total.loc[i+1, 'R']
        if r1 > r2:
            results.extend([1, 0])
        elif r1 < r2:
            results.extend([0, 1])
        else:
            results.extend([-1, -1])
        home_away.extend([0, 1])
    df_total['Result'] = results
    df_total['home_away'] = home_away

    # 무승부 제거 (두 줄 모두 제거)
    draw_indices = df_total[df_total['Result'] == -1].index
    paired_draw_indices = draw_indices.union(draw_indices + (-1) ** (draw_indices % 2 == 0))
    df_total = df_total.drop(paired_draw_indices).reset_index(drop=True)

    # GameID 제거, GameDate를 맨 앞으로
    df_total.drop(columns=['GameID'], inplace=True)
    cols = ['GameDate'] + [col for col in df_total.columns if col != 'GameDate']
    df_total = df_total[cols]

    # 팀 이름 숫자로 인코딩
    team_mapping = {
        'KIA': 0, '두산': 1, '삼성': 2, '키움': 3, '롯데': 4,
        'SSG': 5, '한화': 6, 'KT': 7, 'LG': 8, 'NC': 9
    }
    df_total['Team'] = df_total['Team'].replace(team_mapping)

    df_total = df_total.sort_values('GameDate', kind='stable').reset_index(drop=True)

    # 저장
    df_total.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 저장 완료: {output_path}")

# 사용 예시
file_paths = ['2025_03_total.csv', '2025_04_total.csv', '2025_05_total.csv']  # 또는 실제 경로로 변경
output_path = 'processed_baseball_2025.csv'
preprocess_baseball_data_and_save(file_paths, output_path)


## csv 파일 concat 코드

연도별로 모은 csv 파일을 concat

In [ ]:
def preprocess_baseball_data_and_save(file_paths, output_path):
    # 여러 csv 파일을 읽어 하나의 DataFrame으로 합침
    df_total = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

    df_total.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 저장 완료: {output_path}")

# 사용 예시
file_paths = ['processed_baseball_2024.csv', 'processed_baseball_2025.csv']  # 또는 실제 경로로 변경
output_path = 'processed_2425.csv'
preprocess_baseball_data_and_save(file_paths, output_path)

## rolling & 최근 전적 column 생성 코드

rolling(5)를 통해 미래의 야구 승패도 rolling을 사용하여 feature을 만들어 적용할 수 있게 함.
최근 전적을 비교하여 야구 승패를 보다 쉽게 예측 가능

In [ ]:
import pandas as pd

def preprocess_and_save_rolling_only(file_paths, output_path):
    # 파일 합치기
    df_total = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

    # 최근 5경기 승/패 전적 계산
    recent_wins = []
    recent_losses = []
    match_history = {}

    for i in range(0, len(df_total), 2):
        row1 = df_total.iloc[i]
        row2 = df_total.iloc[i + 1]

        t1, t2 = row1['Team'], row2['Team']
        key = tuple(sorted((t1, t2)))

        history = match_history.get(key, [])
        wins_t1 = history.count((t1, 1))
        wins_t2 = history.count((t2, 1))

        recent_wins.extend([wins_t1, wins_t2])
        recent_losses.extend([wins_t2, wins_t1])

        if row1['Result'] == 1:
            history.append((t1, 1))
        elif row2['Result'] == 1:
            history.append((t2, 1))

        match_history[key] = history[-5:]

    df_total['Recent_5_Win'] = recent_wins
    df_total['Recent_5_Loss'] = recent_losses

    # 수치형 컬럼에서 제외할 항목
    numeric_cols = df_total.select_dtypes(include='number').columns.tolist()
    exclude_cols = ['Recent_5_Win', 'Recent_5_Loss'] # 여기에 제외할 열 이름 적으면 됨
    rolling_cols = [col for col in numeric_cols if col not in exclude_cols]

    # 팀별 rolling 평균
    df_rolled = (
        df_total.groupby('Team')[rolling_cols]
        .rolling(window=5, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )

    df_rolled.columns = [f"{col}" for col in df_rolled.columns]

    # 필요한 컬럼만 포함
    df_base = df_total[['GameDate', 'Team', 'Recent_5_Win', 'Recent_5_Loss']].copy()
    df_result = pd.concat([df_base.reset_index(drop=True), df_rolled.reset_index(drop=True)], axis=1)

    # CSV로 저장
    df_result.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 저장 완료: {output_path}")

# 사용 예시
file_paths = ['processed_2122.csv']
output_path = 'rolling_only.csv'
preprocess_and_save_rolling_only(file_paths, output_path)


## 상관 행렬 계산

상관 행렬을 보며 상관 계수가 높은 column을 1개만 남기고 제거하여 ML시 효율적으로 train 할 수 있도록 함

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# CSV 파일 불러오기
df = pd.read_csv('processed_baseball_2021_encoding.csv')

# 상관행렬 계산 (숫자형 데이터만)
correlation_matrix = df.corr(numeric_only=True)

# 히트맵 시각화
plt.figure(figsize=(20, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix of Baseball Data')
plt.tight_layout()
plt.show()

## 다중공선성 계산

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# CSV 파일 불러오기
df = pd.read_csv('processed_baseball_2021_encoding.csv')

# 숫자형 변수만 선택
numeric_df = df.select_dtypes(include=['float64', 'int64']).dropna()

# 상수항 추가
X = add_constant(numeric_df)

# VIF 계산
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# 결과 출력
print(vif_data)

## 특정 column 삭제 코드

correalation matrix와 다중공선성을 계산한 후 필요없는 column들을 삭제

In [ ]:
import pandas as pd

# 1. CSV 불러오기
df = pd.read_csv('your_file.csv')  # ← 여기에 실제 파일명 입력

# 2. 삭제할 컬럼 목록 지정
columns_to_drop = ['삭제할컬럼1', '삭제할컬럼2']  # 예: ['Result', 'NP_total']

# 3. 컬럼 삭제
df.drop(columns=columns_to_drop, inplace=True)

# 4. 결과를 새 파일로 저장
df.to_csv('your_file_modified.csv', index=False, encoding='utf-8-sig')  # ← 저장 파일명 지정

print("✅ 컬럼 삭제 및 저장 완료!")